##  1. Imports

In [2]:
import requests
import numpy as np
import json
import pandas as pd
import sys
import collections
import scipy
import matplotlib.pyplot as plt
from PIL import Image
import ibmseti
import os,sys
sys.path.append('/home/cs341seti/cs341-ibm-seti/')
import numpy as np
import pandas as pd
import ibmseti
import collections
import scipy.io
import matplotlib.pyplot as plt
import commonutils as cu
import PIL
from PIL import Image
import model_specs

Using TensorFlow backend.


## 2. Gathered the File list store in a data frame

In [3]:
# Downloading and saving all files to local
r = requests.get('https://dal.objectstorage.open.softlayer.com/v1/AUTH_cdbef52bdf7a449c96936e1071f0a46b/simsignals_files/public_list_headers_3april_2017.txt')
fileList = r.text.split('\n')[:-1]
fileListJSON = [json.loads(entry) for entry in fileList]
# Create a data frame with all the file info
fileListTuples = [(str(j['file_name']),str(j['signal_classification']),str(j['uuid'])) for j in fileListJSON]
fileListDF = pd.DataFrame.from_records(fileListTuples,columns=["file_name","signal_classification","uuid"])
# Creating a file index. Will use this for filenames for easier reference
fileListDF['file_index'] = fileListDF.index
fileListDF['file_index'] = fileListDF.file_index.apply(lambda x: str.zfill(str(x),6) )
## Assigning numbers to classes
labelMap = {'noise':0,'brightpixel':1,'narrowband':2,
            'narrowbanddrd':3,'squarepulsednarrowband':4,'squiggle':5,'squigglesquarepulsednarrowband':6}
fileListDF["label"] = fileListDF["signal_classification"].apply(lambda x: labelMap[x])
print labelMap
fileListDF.to_csv("fileList.csv",index=False)

{'noise': 0, 'squiggle': 5, 'narrowbanddrd': 3, 'brightpixel': 1, 'narrowband': 2, 'squigglesquarepulsednarrowband': 6, 'squarepulsednarrowband': 4}


## 3a. Functionality to download and save the Welch periodogram as a numpy array

In [6]:
# Function: input - JSON line with url, output - spectrogram array
def downloadSaveSignalData(DFrow,base_url,container,data_path):
    #********************Configuration Parameters Start************************#
    # This is the expected pixel width of the output image, height is determined automatically
    desiredSignalWidth = 512
    assert desiredSignalWidth&31 == 0 , "Desired Image width should be a multiple of 32"
    # How much should should the periodogram frequency be scaled by determines periodogram length
    periodogramFrequencyScaling = 4
    # rotatePSD - accounts for the 90 degree rotation caused by the Welch Periodogram
    rotatePsd = True
    # Normalize PSD
    NormalizePsd = True
    # Gauss Filter PSD
    GaussFilterPsd = True
    # Number of columns eliminated fromthe periodogram to preven the vertical line around the center frequency
    numEliminatedColumns = 2
    #********************Configuration Parameters End**************************#
    # Access file via HTTP method
    fname = DFrow['file_name']
    print "\r{}".format(DFrow["file_index"]),
    r = requests.get('{}/{}/{}'.format(base_url, container, fname))   
    aca = ibmseti.compamp.SimCompamp(r.content)
    originalSignal = aca.get_signal()
    # 2. Get the signal dimensions
    rawSignalLength = originalSignal.shape[0]
    # 3. Finding the closest multiple of 32 for final image width
    desiredSignalWidth = ((desiredSignalWidth)&(~31))
    #  This is also the segment length of the periodoogram(num frequency bins)
    npersegIn = desiredSignalWidth
    # 4. Signal width as determined by the frequency scaling needed for noise rejection
    signalWidth = desiredSignalWidth * periodogramFrequencyScaling+numEliminatedColumns
    # Finding the scaled signal length to accommodate width as a multiple of desiredSignalWidth*periodogramFrequencyScaling and height as a multiple of 32
    scaledSignalLength = signalWidth * ((rawSignalLength/signalWidth)&(~31))
    #print(scaledSignalLength)
    signalHeight = scaledSignalLength/signalWidth
    scaledSignal = originalSignal[0:scaledSignalLength]
    # 5. Reshaping the scaled signal
    reshapedSignal = aca._reshape(scaledSignal,(signalHeight,signalWidth))
    # 6. Computing the periodogoram
    from scipy import signal
    welchPeriodogram = signal.welch(reshapedSignal,fs= 8000, nperseg=npersegIn,return_onesided=False)
    powerSpectralDensity = welchPeriodogram[1]
    # 7. Rotating the periodogoram + gauss filtering + normalizing
    rotatedPSD = np.hstack((powerSpectralDensity[:,256:(desiredSignalWidth - numEliminatedColumns/2)],powerSpectralDensity[:,numEliminatedColumns/2:desiredSignalWidth/2]))
    rotatedPSDGaussFiltered = rotatedPSD
    if(GaussFilterPsd):
        rotatedPSDGaussFiltered = scipy.ndimage.filters.gaussian_filter(rotatedPSD,sigma=1,order=0,mode="nearest")        
    rotatedNormalizedPSDGaussFiltered = rotatedPSDGaussFiltered
    if(NormalizePsd):
        rotatedNormalizedPSDGaussFiltered = rotatedPSDGaussFiltered/np.linalg.norm(rotatedPSDGaussFiltered)
    np.save("{}/{}.npy".format(data_path,DFrow['file_index']),rotatedNormalizedPSDGaussFiltered) 

## 3b. Download all periodogram data, save as a set of numpy arrays (skip if already exists)

In [7]:
base_url = 'https://dal.objectstorage.open.softlayer.com/v1/AUTH_cdbef52bdf7a449c96936e1071f0a46b'
container = 'simsignals'
data_path = "../data/specdataraw"
### Iterate through fileListDF and save the periodogram of the signal data as .npy file
### Use file_index for filename
temp = fileListDF.apply(downloadSaveSignalData,axis=1,args=(base_url,container,data_path))
del temp

008187                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

015984                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

## 4. Converting all periodograms(numpy files) to images (skip if already exists)

In [5]:
#********************Configuration Parameters Start************************#
numEliminatedColumns = 2
desiredImageWidth = 224
desiredImageHeight = 224
overwriteFiles = False
#********************Configuration Parameters End**************************#
specPath = "../data/specdataraw/"
specfiles = [f for f in os.listdir(specPath) if os.path.isfile(os.path.join(specPath, f))]
for i in range(len(specfiles)):
    fname = specfiles[i].split(".")[0]
    savePath = "../data/specdataimages_{}x{}/".format(desiredImageWidth,desiredImageHeight)
    if(not os.path.isfile('{}{}.jpeg'.format(savePath,fname))):
        # Convert and save each file as image
        print('Creating {}{}.jpeg\n'.format(savePath,fname))
        spec = np.load("{}{}.npy".format(specPath,fname))
        fig, ax = plt.subplots(figsize=(20, 10))
        ax.axis('off')
        ax.frameon = False
        fig.frameon = False
        ax.imshow((spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0],cmap="viridis",interpolation='nearest')
        # Saving the plot
        print("{}{}.png".format(savePath,fname))
        plt.savefig("{}{}.png".format(savePath,fname))
        # reopening 
        from PIL import Image
        img = Image.open("{}{}.png".format(savePath,fname))
        width = img.size[0]
        height = img.size[1]
        cropped_img = img.crop((200, 100, width-200, height-100))
        cropped_img = cropped_img.resize((desiredImageWidth,desiredImageWidth), Image.LANCZOS)
        cropped_img.save('{}{}.jpeg'.format(savePath,fname))
        plt.close('all')
    else:
        print('{}{}.jpeg Exists, skipping \n'.format(savePath,fname))
        

../data/specdataimages_224x224/015545.jpeg Exists, skipping 

../data/specdataimages_224x224/006933.jpeg Exists, skipping 

../data/specdataimages_224x224/007491.jpeg Exists, skipping 

../data/specdataimages_224x224/000525.jpeg Exists, skipping 

../data/specdataimages_224x224/012655.jpeg Exists, skipping 

../data/specdataimages_224x224/001594.jpeg Exists, skipping 

../data/specdataimages_224x224/014279.jpeg Exists, skipping 

../data/specdataimages_224x224/009950.jpeg Exists, skipping 

../data/specdataimages_224x224/012482.jpeg Exists, skipping 

../data/specdataimages_224x224/007872.jpeg Exists, skipping 

../data/specdataimages_224x224/008652.jpeg Exists, skipping 

../data/specdataimages_224x224/003776.jpeg Exists, skipping 

../data/specdataimages_224x224/013965.jpeg Exists, skipping 

../data/specdataimages_224x224/011441.jpeg Exists, skipping 

../data/specdataimages_224x224/015813.jpeg Exists, skipping 

../data/specdataimages_224x224/010644.jpeg Exists, skipping 

../data/

../data/specdataimages_224x224/004417.jpeg Exists, skipping 

../data/specdataimages_224x224/006383.jpeg Exists, skipping 

../data/specdataimages_224x224/010892.jpeg Exists, skipping 

../data/specdataimages_224x224/013794.jpeg Exists, skipping 

../data/specdataimages_224x224/007476.jpeg Exists, skipping 

../data/specdataimages_224x224/001893.jpeg Exists, skipping 

../data/specdataimages_224x224/006231.jpeg Exists, skipping 

../data/specdataimages_224x224/008867.jpeg Exists, skipping 

../data/specdataimages_224x224/011379.jpeg Exists, skipping 

../data/specdataimages_224x224/006429.jpeg Exists, skipping 

../data/specdataimages_224x224/001265.jpeg Exists, skipping 

../data/specdataimages_224x224/004954.jpeg Exists, skipping 

../data/specdataimages_224x224/014477.jpeg Exists, skipping 

../data/specdataimages_224x224/001257.jpeg Exists, skipping 

../data/specdataimages_224x224/015376.jpeg Exists, skipping 

../data/specdataimages_224x224/003332.jpeg Exists, skipping 

../data/

../data/specdataimages_224x224/006257.jpeg Exists, skipping 

../data/specdataimages_224x224/015106.jpeg Exists, skipping 

../data/specdataimages_224x224/014963.jpeg Exists, skipping 

../data/specdataimages_224x224/011799.jpeg Exists, skipping 

../data/specdataimages_224x224/015015.jpeg Exists, skipping 

../data/specdataimages_224x224/001208.jpeg Exists, skipping 

../data/specdataimages_224x224/009547.jpeg Exists, skipping 

../data/specdataimages_224x224/006854.jpeg Exists, skipping 

../data/specdataimages_224x224/011657.jpeg Exists, skipping 

../data/specdataimages_224x224/002797.jpeg Exists, skipping 

../data/specdataimages_224x224/008726.jpeg Exists, skipping 

../data/specdataimages_224x224/001914.jpeg Exists, skipping 

../data/specdataimages_224x224/004399.jpeg Exists, skipping 

../data/specdataimages_224x224/003937.jpeg Exists, skipping 

../data/specdataimages_224x224/011241.jpeg Exists, skipping 

../data/specdataimages_224x224/014458.jpeg Exists, skipping 

../data/

Creating ../data/specdataimages_224x224/011951.jpeg

../data/specdataimages_224x224/011951.png
Creating ../data/specdataimages_224x224/008343.jpeg

../data/specdataimages_224x224/008343.png
Creating ../data/specdataimages_224x224/008082.jpeg

../data/specdataimages_224x224/008082.png
Creating ../data/specdataimages_224x224/008568.jpeg

../data/specdataimages_224x224/008568.png
Creating ../data/specdataimages_224x224/013031.jpeg

../data/specdataimages_224x224/013031.png
Creating ../data/specdataimages_224x224/012371.jpeg

../data/specdataimages_224x224/012371.png
Creating ../data/specdataimages_224x224/009088.jpeg

../data/specdataimages_224x224/009088.png
Creating ../data/specdataimages_224x224/008406.jpeg

../data/specdataimages_224x224/008406.png
Creating ../data/specdataimages_224x224/002004.jpeg

../data/specdataimages_224x224/002004.png
Creating ../data/specdataimages_224x224/011977.jpeg

../data/specdataimages_224x224/011977.png
Creating ../data/specdataimages_224x224/008894.jpe

Creating ../data/specdataimages_224x224/006498.jpeg

../data/specdataimages_224x224/006498.png
Creating ../data/specdataimages_224x224/011248.jpeg

../data/specdataimages_224x224/011248.png
Creating ../data/specdataimages_224x224/009411.jpeg

../data/specdataimages_224x224/009411.png
Creating ../data/specdataimages_224x224/000339.jpeg

../data/specdataimages_224x224/000339.png
Creating ../data/specdataimages_224x224/013032.jpeg

../data/specdataimages_224x224/013032.png
Creating ../data/specdataimages_224x224/014824.jpeg

../data/specdataimages_224x224/014824.png
Creating ../data/specdataimages_224x224/008918.jpeg

../data/specdataimages_224x224/008918.png
Creating ../data/specdataimages_224x224/010496.jpeg

../data/specdataimages_224x224/010496.png
Creating ../data/specdataimages_224x224/013830.jpeg

../data/specdataimages_224x224/013830.png
Creating ../data/specdataimages_224x224/003276.jpeg

../data/specdataimages_224x224/003276.png
Creating ../data/specdataimages_224x224/004157.jpe

...

Creating ../data/specdataimages_224x224/008399.jpeg

../data/specdataimages_224x224/008399.png
Creating ../data/specdataimages_224x224/001344.jpeg

../data/specdataimages_224x224/001344.png
Creating ../data/specdataimages_224x224/001929.jpeg

../data/specdataimages_224x224/001929.png
Creating ../data/specdataimages_224x224/002346.jpeg

../data/specdataimages_224x224/002346.png
Creating ../data/specdataimages_224x224/010065.jpeg

../data/specdataimages_224x224/010065.png
Creating ../data/specdataimages_224x224/000677.jpeg

../data/specdataimages_224x224/000677.png
Creating ../data/specdataimages_224x224/000041.jpeg

../data/specdataimages_224x224/000041.png
Creating ../data/specdataimages_224x224/000246.jpeg

../data/specdataimages_224x224/000246.png
Creating ../data/specdataimages_224x224/000486.jpeg

../data/specdataimages_224x224/000486.png
Creating ../data/specdataimages_224x224/008208.jpeg

../data/specdataimages_224x224/008208.png
Creating ../data/specdataimages_224x224/000154.jpe

Creating ../data/specdataimages_224x224/014237.jpeg

../data/specdataimages_224x224/014237.png
Creating ../data/specdataimages_224x224/003472.jpeg

../data/specdataimages_224x224/003472.png
Creating ../data/specdataimages_224x224/011045.jpeg

../data/specdataimages_224x224/011045.png
Creating ../data/specdataimages_224x224/004050.jpeg

../data/specdataimages_224x224/004050.png
Creating ../data/specdataimages_224x224/008561.jpeg

../data/specdataimages_224x224/008561.png
Creating ../data/specdataimages_224x224/006213.jpeg

../data/specdataimages_224x224/006213.png
Creating ../data/specdataimages_224x224/001236.jpeg

../data/specdataimages_224x224/001236.png
Creating ../data/specdataimages_224x224/011286.jpeg

../data/specdataimages_224x224/011286.png
Creating ../data/specdataimages_224x224/008974.jpeg

../data/specdataimages_224x224/008974.png
Creating ../data/specdataimages_224x224/003354.jpeg

../data/specdataimages_224x224/003354.png
Creating ../data/specdataimages_224x224/002745.jpe

Creating ../data/specdataimages_224x224/009774.jpeg

../data/specdataimages_224x224/009774.png
Creating ../data/specdataimages_224x224/012415.jpeg

../data/specdataimages_224x224/012415.png
Creating ../data/specdataimages_224x224/004800.jpeg

../data/specdataimages_224x224/004800.png
Creating ../data/specdataimages_224x224/005237.jpeg

../data/specdataimages_224x224/005237.png
Creating ../data/specdataimages_224x224/014417.jpeg

../data/specdataimages_224x224/014417.png
Creating ../data/specdataimages_224x224/007263.jpeg

../data/specdataimages_224x224/007263.png
Creating ../data/specdataimages_224x224/014550.jpeg

../data/specdataimages_224x224/014550.png
Creating ../data/specdataimages_224x224/000320.jpeg

../data/specdataimages_224x224/000320.png
Creating ../data/specdataimages_224x224/002260.jpeg

../data/specdataimages_224x224/002260.png
Creating ../data/specdataimages_224x224/001220.jpeg

../data/specdataimages_224x224/001220.png
Creating ../data/specdataimages_224x224/000671.jpe

Creating ../data/specdataimages_224x224/013598.jpeg

../data/specdataimages_224x224/013598.png
Creating ../data/specdataimages_224x224/003773.jpeg

../data/specdataimages_224x224/003773.png
Creating ../data/specdataimages_224x224/008775.jpeg

../data/specdataimages_224x224/008775.png
Creating ../data/specdataimages_224x224/008363.jpeg

../data/specdataimages_224x224/008363.png
Creating ../data/specdataimages_224x224/001546.jpeg

../data/specdataimages_224x224/001546.png
Creating ../data/specdataimages_224x224/007001.jpeg

../data/specdataimages_224x224/007001.png
Creating ../data/specdataimages_224x224/014784.jpeg

../data/specdataimages_224x224/014784.png
Creating ../data/specdataimages_224x224/011763.jpeg

../data/specdataimages_224x224/011763.png
Creating ../data/specdataimages_224x224/006759.jpeg

../data/specdataimages_224x224/006759.png
Creating ../data/specdataimages_224x224/002049.jpeg

../data/specdataimages_224x224/002049.png
Creating ../data/specdataimages_224x224/015112.jpe

Creating ../data/specdataimages_224x224/000296.jpeg

../data/specdataimages_224x224/000296.png
Creating ../data/specdataimages_224x224/002320.jpeg

../data/specdataimages_224x224/002320.png
Creating ../data/specdataimages_224x224/014060.jpeg

../data/specdataimages_224x224/014060.png
Creating ../data/specdataimages_224x224/008299.jpeg

../data/specdataimages_224x224/008299.png
Creating ../data/specdataimages_224x224/007951.jpeg

../data/specdataimages_224x224/007951.png
Creating ../data/specdataimages_224x224/007433.jpeg

../data/specdataimages_224x224/007433.png
Creating ../data/specdataimages_224x224/013319.jpeg

../data/specdataimages_224x224/013319.png
Creating ../data/specdataimages_224x224/001434.jpeg

../data/specdataimages_224x224/001434.png
Creating ../data/specdataimages_224x224/010242.jpeg

../data/specdataimages_224x224/010242.png
Creating ../data/specdataimages_224x224/005882.jpeg

../data/specdataimages_224x224/005882.png
Creating ../data/specdataimages_224x224/000857.jpe

## 5. Generating activations from images using VGG net

In [ ]:
# Point to directory
# Call model activations function
readDir = "../data/specdataimages_224x224_rgb_periodogram/"
writeDir = "../data/specdataactivations_224x224_rgb_periodogram/"
layerName = "block5_pool"
poolfitSize = None
cu.datautils.generateAllActivations(readDir,writeDir,layerName,poolfitSize)

File 8 / 15985: 014906.jpeg

commonutils/datautils.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  model_selection = Model(input=model.input, output=model.get_layer(layer_name).output)


File 1936 / 15985: 001595.jpeg                                                                                                                                                                                                                                              